In [69]:
import os

In [70]:
api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# Importing the necessary packages
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain import PromptTemplate
from langchain import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import os
from langchain_openai import OpenAIEmbeddings

In [51]:
prompt = """
### System:
You are an AI Assistant that generates SQL query for querying SQL data that is distributed in 4 separate sql files and which 
are connected by LINE_KEY   /
ASSET is the main csv and it contains 'assetTRANSACTION_ID', 'assetTRANSACTION_LINE_ID', 'LINE_KEY', /
       'assetCLIENT_NAME', 'assetCLIENT_GROUP', 'assetASSET_REFERENCE_ID', /
       'assetASSET_NAME', 'assetASSET_CATEGORY', 'assetASSET_FAMILY', /
       'assetASSET_SUBFAMILY', 'assetASSET_UNIT_WEIGHT_KG', /
       'assetASSET_DESCRIPTION', 'assetBATCH_SIZE', 'assetASSET_SUPPLIERS', /
       'assetASSET_SUPPLIERS_COUNT', 'YEAR' columns / 
ASSET MATERIALS is another csv file that contains 'LINE_KEY', 'asset_matLAYER1_MATERIAL_NAME', /
       'asset_matLAYER1_COMPOSED_PERCENTAGE', 'asset_matLAYER2_MATERIAL_NAME', /
       'asset_matLAYER2_COMPOSED_PERCENTAGE', 'asset_matRAW_MATERIAL_NAME', /
       'asset_matRAW_MATERIAL_NAME_CLEAN', 'asset_matRAW_MATERIAL_COMPOSED_PERCENTAGE', 'asset_matEMISSION_FACTOR', /
       'asset_matUNIT', 'asset_matPERCENTAGE_TOTAL_CONTRIBUTION', /
       'asset_matPART', 'asset_matCLIENT_NAME', 'asset_matCLIENT_GROUP', /
       'asset_matDESCRIPTION', 'YEAR' columns /
CAT4 SAMPLE is a csv file that contains 'cat4FULFILL_DATE', 'cat4CF_ID', 'cat4CF_NAME', 'cat4TRANSPORT_MODE', /
       'cat4INCOTERM', 'cat4LOADING_PORT', 'cat4DISCHARGING_PORT', 'cat4LOADING_COUNTRY', 'cat4DISCHARGING_COUNTRY', /
       'cat4CLIENT', 'cat4SUPPLIER', 'LINE_KEY', 'cat4id_CARTON', 'cat4consolidated_fulfillment_CARTON', /
       'cat4CUSTRECORD_CARTON_INFO_ITEM', 'cat4NUMBER_OF_CARTONS', 'cat4GROSS_WEIGHT_PER_CARTON', /
       'cat4WEIGHT_OF_FREIGHT_KG', 'cat4size_length_CARTON', 'cat4size_width_CARTON', /
       'cat4size_height_CARTON', 'cat4TOTAL_CBM', 'cat4cbm_total_CARTON', 'cat4SHIPPED_QTY', 'cat4SUB_TRANSPORT_MODE', /
       'cat4DISTANCE_TRAVELLED_KM', 'YEAR' columns /
CAT9 SAMPLE is a csv file that contains 'cat9FULFILL_DATE', 'cat9CF_ID', 'cat9CF_NAME', 'cat9TRANSPORT_MODE', /
       'cat9INCOTERM', 'cat9LOADING_PORT', 'cat9DISCHARGING_PORT', /
       'cat9LOADING_COUNTRY', 'cat9DISCHARGING_COUNTRY', 'cat9CLIENT', /
       'cat9SUPPLIER', 'LINE_KEY', 'cat9id_CARTON', /
       'cat9consolidated_fulfillment_CARTON', /
       'cat9CUSTRECORD_CARTON_INFO_ITEM', 'cat9NUMBER_OF_CARTONS', /
       'cat9GROSS_WEIGHT_PER_CARTON', 'cat9WEIGHT_OF_FREIGHT_KG', /
       'cat9size_length_CARTON', 'cat9size_width_CARTON', /
       'cat9size_height_CARTON', 'cat9TOTAL_CBM', 'cat9cbm_total_CARTON', /
       'cat9SHIPPED_QTY', 'cat9SUB_TRANSPORT_MODE', /
       'cat9DISTANCE_TRAVELLED_KM', 'YEAR' columns / 
       all this csv files are connected by same 'assetLINE_KEY', 'asset_matLINE_KEY', 'cat4LINE_KEY', 'cat9LINE_KEY' /
       after i ask you question generate sql query 

### User:
{prompt}

### Response:

"""

In [52]:
template = """
### System:
You are an very helpful assistant. You have to answer the user's. If you don't know the answer, \
just say you don't know. Don't try to make up an answer. and answer as if you are very helpful

### Context:
{context}

### User:
{question}

### Response:
"""

In [53]:
#working
# storing multiple documents in list 
print("Loading data...")
csv_folder_path = "documents"
print(os.listdir(csv_folder_path))

# Load multiple files
loaders = [CSVLoader(os.path.join(csv_folder_path, fn)) for fn in os.listdir(csv_folder_path)]

print(loaders)

all_documents = []

#chunks
for loader in loaders:
    print("Loading raw document..." + loader.file_path)
    raw_documents = loader.load()

    print("Splitting text...")
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=800,
        chunk_overlap=100,
        length_function=len,
    )
    documents = text_splitter.split_documents(raw_documents)
    all_documents.extend(documents)

print("Creating vectorstore...")
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(all_documents, embeddings)

#with open("vectorstore.pkl", "wb") as f:
#    pickle.dump(vectorstore, f)

Loading data...
['asset - Copy.csv', 'asset material - Copy.csv', 'cat4_sample (1) - Copy.csv', 'cat9_sample (1) - Copy.csv']
[<langchain_community.document_loaders.csv_loader.CSVLoader object at 0x000001B308001820>, <langchain_community.document_loaders.csv_loader.CSVLoader object at 0x000001B308001EE0>, <langchain_community.document_loaders.csv_loader.CSVLoader object at 0x000001B307FE3D70>, <langchain_community.document_loaders.csv_loader.CSVLoader object at 0x000001B307FE0A10>]
Loading raw document...documents\asset - Copy.csv
Splitting text...
Loading raw document...documents\asset material - Copy.csv
Splitting text...
Loading raw document...documents\cat4_sample (1) - Copy.csv
Splitting text...
Loading raw document...documents\cat9_sample (1) - Copy.csv
Splitting text...
Creating vectorstore...


In [54]:
print(loader.load()[0:1000])

[Document(page_content='cat9FULFILL_DATE: 2023-01-04T00:00:00Z\ncat9CF_ID: 3731\ncat9CF_NAME: CF-HK-23010001\ncat9TRANSPORT_MODE: Sea\ncat9INCOTERM: FCA\ncat9LOADING_PORT: Yantian, China\ncat9DISCHARGING_PORT: Houston, United States\ncat9LOADING_COUNTRY: China\ncat9DISCHARGING_COUNTRY: United States\ncat9CLIENT: Mary Kay Inc.\ncat9SUPPLIER: DONGGUAN JINGWEI IMPORT AND EXPORT CO.,LTD.\nLINE_KEY: 2870_1656852282431_28470\ncat9id_CARTON: 6762\ncat9consolidated_fulfillment_CARTON: 3731\ncat9CUSTRECORD_CARTON_INFO_ITEM: 7884\ncat9NUMBER_OF_CARTONS: 1\ncat9GROSS_WEIGHT_PER_CARTON: 4.21\ncat9WEIGHT_OF_FREIGHT_KG: 4.21\ncat9size_length_CARTON: 51.5\ncat9size_width_CARTON: 37\ncat9size_height_CARTON: 27\ncat9TOTAL_CBM: 0.0514\ncat9cbm_total_CARTON: 0.0514485\ncat9SHIPPED_QTY: 40\ncat9SUB_TRANSPORT_MODE: Sea\ncat9DISTANCE_TRAVELLED_KM: 20086.79\nYEAR: 2023', metadata={'source': 'documents\\cat9_sample (1) - Copy.csv', 'row': 0}), Document(page_content='cat9FULFILL_DATE: 2023-01-04T00:00:00Z\ncat

In [55]:
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever, 
        chain_type="stuff",
        return_source_documents=True, 
        chain_type_kwargs={'prompt': prompt}
    )

In [56]:
def get_response(query, chain):
    response = chain({'query': query})    
    print(response['result'])

In [57]:
from langchain.llms import Ollama


In [58]:
llm = OpenAI(openai_api_key=api_key)

In [59]:
import os
from llama_index.core import SimpleDirectoryReader
import tempfile

In [60]:
#docs = load_documents(file_path="documents") 
#documents = split_documents(documents=docs)
#vectorstore = create_embeddings(documents, embed_model)

retriever = vectorstore.as_retriever()

In [61]:
prompt = PromptTemplate.from_template(template)

chain = load_qa_chain(retriever, llm, prompt)


In [62]:
get_response("give me information about LINE_KEY that had most emissions and please give me information about assets for same LINE_KEY", chain)

I'm sorry, I don't have that information. Is there something else I can help you with?


In [63]:
get_response("check the assetASSET_SUPPLIERS_COUNT table to see which supplier has the most entries for line key 15447_1666324047578_10216 in 2024 and what are the raw materials they used?", chain)

I am sorry, I am not familiar with that information. However, I can help you find the answer. Would you like me to search for the assetASSET_SUPPLIERS_COUNT table for you?


In [72]:
get_response("give me sql query for finding out all information about LINE_KEY 3173_1666161196360_40382", chain)

I'm sorry, but I don't know the specific SQL query for that LINE_KEY. However, I can help you gather all the relevant information you need for that LINE_KEY. Would you like me to retrieve that information for you?


In [65]:
get_response("what was the total cat4DISTANCE_TRAVELLED_KM for LINE_KEY 3173_1666161196360_3563 and please provide me information about assetASSET_NAME for the same LINE_KEY , please find it for me", chain)

The total cat4DISTANCE_TRAVELLED_KM for LINE_KEY 3173_1666161196360_3563 is 7959 kilometers. As for the assetASSET_NAME, I'm sorry, I don't know the specific information for that asset. Is there something else I can assist you with?


In [66]:
get_response("what is the year for transaction and what is cat9TRANSPORT_MODE of LINE_KEY 2870_1688112309623_90101", chain)

The year for this transaction is 2023 and the transport mode for cat9 is Sea. Is there anything else I can assist you with?


In [67]:
get_response("i need all details about LINE_KEY 3173_1666161196360_21703 ", chain)

I'm happy to assist you with that! The LINE_KEY 3173_1666161196360_21703 refers to a transaction line for a bracelet under the fashion accessories category. It has a batch size of 1196 and is supplied by two manufacturers, Kunshan ZhouZhuang fengda garment accessories factory and Chic Accessories Factory. Is there any other information you need about this LINE_KEY?
